# Simple Online and Realtime Tracking
## Reduced notebook only to SORT tracker

In [1]:
import os
import cv2
import glob
import argparse
import numpy as np

In [2]:
# sort contains the SORT traching algorithm and can be downloaded from: 
# https://github.com/AyhamOlk/tracking_characters_scene_friends_series/blob/main/sort.py
from sort import *

<b>To Check before running: </b><br>
Either make sure the results of the detections are stored in the './all_outputs/labels' directory under the
form 002400.txt, 002401.txt, ..., <br>
Otherwise, if stored somewhere else, simply change the directory name when running `change_first_forma`
function [here](#change_dir_name)!<br>
<br>
Typical example of file '002400.txt':<br>
534 196 63 77<br>
271 205 44 79<br>
999 308 42 43<br>
1036 208 54 74<br>

In [5]:
# Check you are the project root
pwd

'/Users/flavienvidal/INF634'

In [6]:
def change_first_format(path_to_final_res: str = "./data/train/which_frames/det/det.txt", 
                        path_to_current_det: str = "./all_outputs/labels"):
    """
    Modifies the initial detection results format to fit into the tracker. The correct "det.txt" format is as 
    follow with a signle comma and no space between the figures: frame_id,-1,xmin,ymin,w,h,confidence,-1,-1,-1
    
    Parameters
    ----------
    path_to_current_det: path to the detections result files (002400.txt, 002401.txt, ...)
        534 196 63 77
        271 205 44 79
        999 308 42 43
        1036 208 54 74
    path_to_final_res: path to save the final detections result files in the appropiate format
        2400,-1,534,196,63,77,1,-1,-1,-1
        2400,-1,271,205,44,79,1,-1,-1,-1
        2400,-1,999,308,42,43,1,-1,-1,-1
        2400,-1,1036,208,54,74,1,-1,-1,-1
    """

    confidence = 1 # todo: set to 1 for now but need to change it later with Ayham's predictions confidences

    studied_frames_res = os.listdir(path_to_current_det)
    studied_frames_res.sort() # typically: studied_frames_res = ['002400.txt', '002401.txt', '002402.txt', ...]

    starting_frame = int(studied_frames_res[0].replace(".txt", ""))
    ending_frame = int(studied_frames_res[-1].replace(".txt", ""))

    path_to_final_res = path_to_final_res.replace("which_frames", f"frames_{starting_frame}_to_{ending_frame}")

    if not os.path.exists(path_to_final_res.replace("det.txt", "")):
        os.makedirs(path_to_final_res.replace("det.txt", ""))
        
    final_det = open(path_to_final_res, "w")
    
    all_files =[]
    for frame in studied_frames_res:
        all_files.append(os.path.join(path_to_current_det, frame))

    for frame in all_files:
        file = open(frame, "r") 
        for line in file.readlines():
            L = line.split(" ")
            frame_id = int(frame.split("/")[-1].split(".txt")[0])
            xmin = L[0]
            ymin = L[1]
            w = L[2]
            h = L[3].replace("\n", "")

            final_det.write('{},{},{},{},{},{},{},{},{},{}\n'.format(frame_id, -1, xmin, ymin, w, h, confidence, -1, -1, -1)) 
    final_det.close()


<a id='change_dir_name'></a>

In [7]:
change_first_format(path_to_final_res = "./data/train/which_frames/det/det.txt", 
                    path_to_current_det = "./all_outputs/labels") # change here the dir to the detection results

In [8]:
# Add all arguments needed for the SORT tracking algorithm

parser = argparse.ArgumentParser(description='SORT')
parser.add_argument('-f')
parser.add_argument('--display', dest='display', help='Display online tracker output (slow) [False]',action='store_true')
parser.add_argument("--seq_path", help="Path to detections.", type=str, default='data')
parser.add_argument("--phase", help="Subdirectory in seq_path.", type=str, default='train')
parser.add_argument("--max_age", help="Maximum number of frames to keep alive a track without associated detections.", type=int, default=1)
parser.add_argument("--min_hits", help="Minimum number of associated detections before track is initialised.", type=int, default=3)
parser.add_argument("--iou_threshold", help="Minimum IOU for match.", type=float, default=0.3)
args = parser.parse_args()


In [9]:
# With display:
args = parser.parse_args("--display --seq_path data --phase train --max_age 1 --min_hits 3 --iou_threshold 0.3".split())

# Without display:
# args = parser.parse_args("--seq_path data --phase train --max_age 1 --min_hits 3 --iou_threshold 0.3".split())

In [10]:
display = args.display
phase = args.phase
colours = np.random.rand(32, 3)

if(display):
    plt.ion()
    fig = plt.figure()
    ax1 = fig.add_subplot()

# Creates an output dir to ...
if not os.path.exists('output'):
    os.makedirs('output')

# pattern is the path to all det.txt detection files
pattern = os.path.join(args.seq_path, phase, '*', 'det', 'det.txt')

print(f"glob.glob(pattern) = {print(glob.glob(pattern))}")
for seq_dets_fn in glob.glob(pattern): # takes all det.txt files one by one
    print(f"seq_dets_fn = {seq_dets_fn}")
    mot_tracker = Sort(max_age=args.max_age, 
                       min_hits=args.min_hits,
                       iou_threshold=args.iou_threshold) # creates instance of the SORT tracker
    seq_dets = np.loadtxt(seq_dets_fn, delimiter=',')
    print(f"seq_dets = {seq_dets}")
    seq = seq_dets_fn[pattern.find('*'):].split(os.path.sep)[0]
    print(f"seq = {seq}")

    with open(os.path.join('output', '%s.txt'%(seq)),'w') as out_file:
        print("Processing %s."%(seq))
        
        previous_det = [[]]
        
        for frame in range(int(seq_dets[:, 0].min())-1, int(seq_dets[:, 0].max())): # modofied vrs
            print(f"\n\u25A0 frame = {frame} (min = {int(seq_dets[:, 0].min())}, max = {int(seq_dets[:,0].max())})")
            frame += 1 #detection and frame numbers begin at 1
            print(f"  seq_dets[:, 0]==frame = {seq_dets[:, 0]==frame}")
            dets = seq_dets[seq_dets[:, 0]==frame, 2:7]
            # Will typically be of the form:
            # dets = [[ 534.  196.   63.   77.    0.]
            #         [ 271.  205.   44.   79.    0.]
            #         [ 999.  308.   42.   43.    0.]
            #         [1036.  208.   54.   74.    0.]]

            print(f"  dets = {dets}")
            dets[:, 2:4] += dets[:, 0:2] # convert initial [x1,y1,w,h] to [x1,y1,x2,y2]
            print(f"  dets = {dets}")

            if(display):
                #fn = os.path.join('mot_benchmark', phase, seq, 'img1', '%06d.jpg'%(frame))
                fn = os.path.join('season1/episode1', '%06d.jpg'%(frame))
                print(f"    \u21FE fn = {fn}")
                im = io.imread(fn)
                ax1.imshow(im)
                plt.title(seq + ' Tracked Targets')

            start_time = time.time()
            trackers = mot_tracker.update(dets)
            print(f"  trackers = {trackers}")
            cycle_time = time.time() - start_time


            for d in trackers:
                
                if d[4] not in previous_det[-1]:
                    # Then crop the corresponding face from the image and pass it through the face_recogniton fun
                    # run face_recognition()
                    
                    x, y, h, w, = 139, 143, 789, 1490
                    img = cv2.imread("det_and_track_results/frames_2400_to_2650/frame_2400.jpg")
                    new_detected_face = im[d[1]:d[3], d[0]:d[2]]
                    pass
                
                previous_det.append(trackers[:, -1])
                    
                    
                print('%d,%d,%.2f,%.2f,%.2f,%.2f,1,-1,-1,-1'%(frame,d[4],d[0],d[1],d[2]-d[0],d[3]-d[1]))
                print('%d,%d,%.2f,%.2f,%.2f,%.2f,1,-1,-1,-1'%(frame,d[4],d[0],d[1],d[2]-d[0],d[3]-d[1]),file=out_file)
                if(display):
                    d = d.astype(np.int32)
                    ax1.add_patch(patches.Rectangle((d[0], d[1]),
                                                    d[2]-d[0],
                                                    d[3]-d[1],
                                                    fill=False,
                                                    lw=2,
                                                    ec=colours[d[4]%32,:], 
                                                    label='label'))
                    ax1.text(d[0]-10, d[1]-10, d[4], color=colours[d[4]%32,:])
                    
            
            if not os.path.exists(f'det_and_track_results/{seq}'):
                os.makedirs(f'det_and_track_results/{seq}')
                
            plt.savefig(f'det_and_track_results/{seq}/frame_{frame}.jpg', dpi=300, bbox_inches='tight')

            if(display):
                fig.canvas.flush_events()
                plt.draw()
                ax1.cla()

    if(display):
        print("Note: to get real runtime results run without the option: --display")


all_images = glob.glob(f'det_and_track_results/{seq}/*.jpg')
all_images.sort()

img_array = []
x, y, h, w, = 139, 143, 789, 1490

for filename in all_images: 
    print(filename)
    img = cv2.imread(filename)
    img = img[y:y+h, x:x+w]
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)
    
if not os.path.exists(f'final_videos/{seq}'):
    os.makedirs(f'final_videos/{seq}')
    
out = cv2.VideoWriter(f'final_videos/video_{seq}.avi', cv2.VideoWriter_fourcc(*'DIVX'), 15, size)

for i in range(len(img_array)):
    out.write(img_array[i])
out.release()



['data/train/frames_2400_to_2650/det/det.txt']
glob.glob(pattern) = None
seq_dets_fn = data/train/frames_2400_to_2650/det/det.txt
seq_dets = [[ 2.40e+03 -1.00e+00  5.34e+02 ... -1.00e+00 -1.00e+00 -1.00e+00]
 [ 2.40e+03 -1.00e+00  2.71e+02 ... -1.00e+00 -1.00e+00 -1.00e+00]
 [ 2.40e+03 -1.00e+00  9.99e+02 ... -1.00e+00 -1.00e+00 -1.00e+00]
 ...
 [ 2.65e+03 -1.00e+00  9.12e+02 ... -1.00e+00 -1.00e+00 -1.00e+00]
 [ 2.65e+03 -1.00e+00  2.25e+02 ... -1.00e+00 -1.00e+00 -1.00e+00]
 [ 2.65e+03 -1.00e+00  1.38e+02 ... -1.00e+00 -1.00e+00 -1.00e+00]]
seq = frames_2400_to_2650
Processing frames_2400_to_2650.

■ frame = 2399 (min = 2400, max = 2650)
  seq_dets[:, 0]==frame = [ True  True  True  True False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False Fal

Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/flavienvidal/opt/anaconda3/lib/python3.8/tkinter/__init__.py", line 1883, in __call__
    return self.func(*args)
  File "/Users/flavienvidal/opt/anaconda3/lib/python3.8/tkinter/__init__.py", line 804, in callit
    func(*args)
  File "/Users/flavienvidal/opt/anaconda3/lib/python3.8/site-packages/matplotlib/backends/_backend_tk.py", line 253, in idle_draw
    self.draw()
  File "/Users/flavienvidal/opt/anaconda3/lib/python3.8/site-packages/matplotlib/backends/backend_tkagg.py", line 9, in draw
    super(FigureCanvasTkAgg, self).draw()
  File "/Users/flavienvidal/opt/anaconda3/lib/python3.8/site-packages/matplotlib/backends/backend_agg.py", line 407, in draw
    self.figure.draw(self.renderer)
  File "/Users/flavienvidal/opt/anaconda3/lib/python3.8/site-packages/matplotlib/backend_bases.py", line 1544, in _draw
    def _draw(renderer): raise Done(renderer)
matplotlib.backend_bases._get_renderer.<locals>.Done


■ frame = 2400 (min = 2400, max = 2650)
  seq_dets[:, 0]==frame = [False False False False  True  True  True  True  True  True  True False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False Fa


■ frame = 2402 (min = 2400, max = 2650)
  seq_dets[:, 0]==frame = [False False False False False False False False False False False False
 False False False False  True  True  True  True  True False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False Fa


■ frame = 2404 (min = 2400, max = 2650)
  seq_dets[:, 0]==frame = [False False False False False False False False False False False False
 False False False False False False False False False False False False
 False  True  True  True False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False Fa

...


■ frame = 2644 (min = 2400, max = 2650)
  seq_dets[:, 0]==frame = [False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False Fa


■ frame = 2646 (min = 2400, max = 2650)
  seq_dets[:, 0]==frame = [False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False Fa


■ frame = 2648 (min = 2400, max = 2650)
  seq_dets[:, 0]==frame = [False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False Fa

Note: to get real runtime results run without the option: --display
det_and_track_results/frames_2400_to_2650/frame_2400.jpg
det_and_track_results/frames_2400_to_2650/frame_2401.jpg
det_and_track_results/frames_2400_to_2650/frame_2402.jpg
det_and_track_results/frames_2400_to_2650/frame_2403.jpg
det_and_track_results/frames_2400_to_2650/frame_2404.jpg
det_and_track_results/frames_2400_to_2650/frame_2405.jpg
det_and_track_results/frames_2400_to_2650/frame_2406.jpg
det_and_track_results/frames_2400_to_2650/frame_2407.jpg
det_and_track_results/frames_2400_to_2650/frame_2408.jpg
det_and_track_results/frames_2400_to_2650/frame_2409.jpg
det_and_track_results/frames_2400_to_2650/frame_2410.jpg
det_and_track_results/frames_2400_to_2650/frame_2411.jpg
det_and_track_results/frames_2400_to_2650/frame_2412.jpg
det_and_track_results/frames_2400_to_2650/frame_2413.jpg
det_and_track_results/frames_2400_to_2650/frame_2414.jpg
det_and_track_results/frames_2400_to_2650/frame_2415.jpg
det_and_track_result

det_and_track_results/frames_2400_to_2650/frame_2544.jpg
det_and_track_results/frames_2400_to_2650/frame_2545.jpg
det_and_track_results/frames_2400_to_2650/frame_2546.jpg
det_and_track_results/frames_2400_to_2650/frame_2547.jpg
det_and_track_results/frames_2400_to_2650/frame_2548.jpg
det_and_track_results/frames_2400_to_2650/frame_2549.jpg
det_and_track_results/frames_2400_to_2650/frame_2550.jpg
det_and_track_results/frames_2400_to_2650/frame_2551.jpg
det_and_track_results/frames_2400_to_2650/frame_2552.jpg
det_and_track_results/frames_2400_to_2650/frame_2553.jpg
det_and_track_results/frames_2400_to_2650/frame_2554.jpg
det_and_track_results/frames_2400_to_2650/frame_2555.jpg
det_and_track_results/frames_2400_to_2650/frame_2556.jpg
det_and_track_results/frames_2400_to_2650/frame_2557.jpg
det_and_track_results/frames_2400_to_2650/frame_2558.jpg
det_and_track_results/frames_2400_to_2650/frame_2559.jpg
det_and_track_results/frames_2400_to_2650/frame_2560.jpg
det_and_track_results/frames_24